# 비동기 요청

aiohttp는 파이썬의 asyncio 라이브러리를 기반으로 하는 비동기 HTTP 클라이언트 및 서버 프레임워크이다.  
단일 스레드 내에서 이벤트 루프를 통해 다수의 네트워크 요청을 병렬로 처리하여 입출력 대기 시간을 효율적으로 관리한다.

`pip install aiohttp`를 통해 설치한다.

In [1]:
import asyncio
import aiohttp

async def main():
    # ClientSession은 연결 풀을 관리하는 객체이다.
    async with aiohttp.ClientSession() as session:
        # 비동기 context manager를 사용하여 세션을 생성한다.
        pass


await main()

### 비동기 클라이언트 요청

클라이언트 세션을 생성한 후 get, post 등 HTTP 메서드에 대응하는 함수를 호출하여 요청을 보낸다.  
요청 시 await 키워드를 사용하여 응답이 올 때까지 제어권을 이벤트 루프에 반환한다.

In [ ]:
async def fetch_status(url):
    async with aiohttp.ClientSession() as session:
        # GET 요청을 비동기적으로 수행한다.
        async with session.get(url) as response:
            # 응답 상태 코드를 반환받는다.
            status = response.status
            print(f"상태 코드: {status}")
            return status

await fetch_status("https://jsonplaceholder.typicode.com/posts")

### 응답 데이터 처리

응답 객체로부터 텍스트, 바이너리, 제이슨 데이터를 비동기적으로 추출한다.  
데이터 추출 메서드인 text(), json(), read() 역시 코루틴이므로 await 키워드를 사용해야 한다.

In [ ]:
async def get_json_data():
    url = "https://jsonplaceholder.typicode.com/posts/1"
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            # 응답 본문을 제이슨 형식으로 파싱한다.
            data = await response.json()
            # 특정 키의 값을 출력한다.
            print(f"제목: {data['title']}")

await get_json_data()

### 다중 요청 병렬 처리

asyncio.gather를 사용하여 여러 개의 aiohttp 요청 태스크를 동시에 실행한다.  
동기 방식의 요청과 달리 각 요청의 완료를 기다리지 않고 다음 요청을 바로 수행한다.

In [ ]:
async def fetch_url(session, url):
    async with session.get(url) as response:
        return await response.text()

async def request_all(urls):
    async with aiohttp.ClientSession() as session:
        # 각 URL에 대한 요청 코루틴 리스트를 생성한다.
        tasks = [fetch_url(session, url) for url in urls]
        # 모든 태스크를 병렬로 실행하고 결과를 수집한다.
        responses = await asyncio.gather(*tasks)
        print(f"응답 개수: {len(responses)}")

url_list = [
    "https://jsonplaceholder.typicode.com/posts/1", 
    "https://jsonplaceholder.typicode.com/posts/2", 
    "https://jsonplaceholder.typicode.com/posts/3", 
    ]
await request_all(url_list)

### 매개변수 및 헤더 전달

요청 시 params 인자를 통해 쿼리 스트링을 전달하거나, headers 인자를 통해 HTTP 헤더를 설정한다.  
딕셔너리 구조를 활용하여 데이터를 정의한다.

In [ ]:
async def search_with_params():
    url = "https://httpbin.org/get"
    # 쿼리 매개변수를 정의한다.
    query_params = {"name": "admin", "id": "123"}
    # 사용자 정의 헤더를 정의한다.
    custom_headers = {"User-Agent": "AiohttpClient/1.0"}

    async with aiohttp.ClientSession() as session:
        async with session.get(url, params=query_params, headers=custom_headers) as response:
            result = await response.json()
            # 서버에서 수신한 인자 정보를 확인한다.
            print(result["args"])

await search_with_params()

### 다중 요청 개수 제한

비동기 환경에서 동시에 너무 많은 네트워크 요청을 보낼 경우 서버 측 차단이나 시스템 자원 고갈이 발생할 수 있다.  
asyncio.Semaphore 객체를 사용하여 동시에 실행 가능한 코루틴의 숫자를 제어한다.

In [ ]:
import asyncio
import aiohttp

async def fetch_with_semaphore(semaphore, session, url):
    # 세마포어를 사용하여 동시 실행 숫자를 제한한다.
    async with semaphore:
        async with session.get(url) as response:
            status = response.status
            # 실제 요청이 수행되는 시점을 확인한다.
            print(f"요청 완료: {url} (상태: {status})")
            return await response.text()

async def main():
    urls = [f"https://jsonplaceholder.typicode.com/posts/{i}" for i in range(1, 21)]
    
    # 동시 요청 숫자를 5개로 제한하는 세마포어를 생성한다.
    semaphore = asyncio.Semaphore(5)
    
    async with aiohttp.ClientSession() as session:
        # 모든 요청 태스크를 생성하되, 세마포어에 의해 5개씩 순차적으로 실행된다.
        tasks = [fetch_with_semaphore(semaphore, session, url) for url in urls]
        await asyncio.gather(*tasks)

await main()

In [ ]:

import requests
from pprint import pprint
import os
from dotenv import load_dotenv

load_dotenv()
BASE_URL = "https://api.themoviedb.org/3"
IMAGE_PATH = "https://image.tmdb.org/t/p/w500/{poster_path}"
tmdb_api_key = os.getenv('TMDB_API_KEY')
API_KEY = tmdb_api_key

# 비동기 호출
async def tmdb_get_async(session: aiohttp.ClientSession, path: str, *, language="ko-KR", **params):
    path = path.lstrip("/")
    url = f"{BASE_URL}/{path}"

    req_params = {"language": language, **params}
    async with session.get(url, params=req_params) as res:
        res.raise_for_status()
        return await res.json()


# tmdb_get
def tmdb_get(path: str, *, api_key = API_KEY, language: str = "ko-KR", **params):
    
    path = path.lstrip("/")
    url = f"{BASE_URL}/{path}"

    headers = {
        "Authorization": f"Bearer {api_key}",
        "accept": "application/json",
    }

    req_params = {"language": language, **params}

    res = requests.get(url, headers=headers, params=req_params, timeout=10)
    res.raise_for_status()
    return res.json()

# 현재 상영중인 영화중 평점이 가장 높은 영화 반환 get_max_vote_average_movie
def get_max_vote_average_movie(movies: list) -> dict:
    max_vote_average = 0
    max_vote_average_movie = None

    for movie in movies:
        if 'vote_average' in movie:
            current_vote = movie['vote_average']
            
            if current_vote > max_vote_average:
                max_vote_average = current_vote
                max_vote_average_movie = movie
                
    return max_vote_average_movie

data = tmdb_get("movie/now_playing")


### 문제
TMDB API를 활용하여 현재 상영 중인 영화(now_playing) 데이터에서 다음 정보만 담긴 리스트를 만드세요.

- title (영화 제목)  
- vote_average (평점)  
- revenue (수익)

In [ ]:
import asyncio
import aiohttp
import os 
from dotenv import load_dotenv

load_dotenv()
tmdb_api_key = os.getenv('TMDB_API_KEY')
API_KEY = tmdb_api_key
now_playing_data = tmdb_get("movie/now_playing", page=1)
response = now_playing_data['results']

# pprint(response)

# 기존 동기 방식
# def fetch_one(movie_id):
#     d = tmdb_get(f"/movie/{movie_id}")
#     return d.get("title"), d.get("vote_average"), d.get("revenue")


# new_lst = []

# for movie in response:
#      now_playing_movie_id = movie.get('id')
#      result = fetch_one(now_playing_movie_id)
#      new_lst.append(result)

# pprint(new_lst)

# 비동기 호출 방식
async def fetch_one(session: aiohttp.ClientSession,
                    movie_id: int,
                    sem: asyncio.Semaphore,
                    *, api_key: str = API_KEY,
                    language: str = "ko-KR",
                    ):
    headers = {
        "Authorization": f"Bearer {api_key}",
        "accept": "application/json",
    }

    async with sem:
        url = f"{BASE_URL}/movie/{movie_id}"
        async with session.get(url, headers=headers, params={"language": language}) as res:
            res.raise_for_status()
            d = await res.json()
            return (d.get("title"), d.get("vote_average"), d.get("revenue"))
        
        
async def fetch_all_details(response, concurrency: int = 10):
    sem = asyncio.Semaphore(concurrency)

    async with aiohttp.ClientSession() as session:
        tasks = []
        for movie in response:
            movie_id = movie.get("id")
            if movie_id is None:
                continue
            tasks.append(fetch_one(session, movie_id, sem))

        results = await asyncio.gather(*tasks, return_exceptions=True)
        
    return results
        
new_lst = await fetch_all_details(response, concurrency=12)

pprint(new_lst)

20
[('더 립', 7.164, 0),
 ('아바타: 불과 재', 7.355, 1320000000),
 ('하우스메이드', 7.189, 245700000),
 ('28년 후: 뼈의 사원', 7.0, 3120000),
 ('포풍추영', 7.16, 174400000),
 ('전지적 독자 시점', 6.7, 9187679),
 ('렌탈 패밀리: 가족을 빌려드립니다', 7.941, 10706787),
 ('프레디의 피자가게 2', 6.766, 236746717),
 ('우리의 열 번째 여름', 7.091, 0),
 ('हैप्पी पटेल: खतरानक जासूस', 7.8, 0),
 ('ปัง', 6.489, 0),
 ('원 배틀 애프터 어나더', 7.433, 206311045),
 ('파과', 6.5, 0),
 ('시수: 복수의 길', 7.429, 9724644),
 ('그린랜드 2', 6.627, 11416907),
 ('마티 슈프림', 8.082, 92925093),
 ('아나콘다', 5.9, 112467421),
 ('극장판 주술회전: 시부야사변 X 사멸회유', 5.628, 44559195),
 ('어쩔수가없다', 7.75, 25343108),
 ('더 러닝 맨', 6.8, 68615641)]
